In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

weights_dict = {}
weight_callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs: weights_dict.update({epoch: model.get_weights()}))


2023-09-25 05:43:24.832172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 05:43:26.212216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize pixel values to [0, 1]
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Modify ResNet-50 for CIFAR-10
input_layer = Input(shape=(32, 32, 3))
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_tensor=input_layer)
x = Flatten()(resnet50_base.output)
x = Dense(256, activation='relu')(x)
output_layer = Dense(10, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

2023-09-25 05:43:33.819301: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11437 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:86:00.0, compute capability: 6.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 38, 38, 3)            0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 16, 16, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [3]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test), callbacks=[weight_callback])

Epoch 1/20


2023-09-25 05:44:06.943965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8905
2023-09-25 05:44:07.132889: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-25 05:44:07.936428: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b3a688e340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-25 05:44:07.936516: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla P100-PCIE-12GB, Compute Capability 6.0
2023-09-25 05:44:07.953805: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-25 05:44:08.084935: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-25 05:44:08.228635: I ./tensorflow/compiler/jit/device_compile

391/391 [==============================] - 98s 100ms/step - loss: 1.0542 - accuracy: 0.6475 - val_loss: 3.8147 - val_accuracy: 0.1000
Epoch 2/20
391/391 [==============================] - 34s 88ms/step - loss: 0.6493 - accuracy: 0.7841 - val_loss: 2.4145 - val_accuracy: 0.3728
Epoch 3/20
391/391 [==============================] - 34s 86ms/step - loss: 0.5162 - accuracy: 0.8257 - val_loss: 0.9981 - val_accuracy: 0.6893
Epoch 4/20
391/391 [==============================] - 35s 89ms/step - loss: 0.4336 - accuracy: 0.8529 - val_loss: 1.3748 - val_accuracy: 0.6030
Epoch 5/20
391/391 [==============================] - 35s 89ms/step - loss: 0.3592 - accuracy: 0.8799 - val_loss: 1.2912 - val_accuracy: 0.6401
Epoch 6/20
391/391 [==============================] - 35s 88ms/step - loss: 0.3022 - accuracy: 0.8981 - val_loss: 0.7777 - val_accuracy: 0.7678
Epoch 7/20
391/391 [==============================] - 34s 88ms/step - loss: 0.2523 - accuracy: 0.9145 - val_loss: 1.3836 - val_accuracy: 0.6690
Ep

In [4]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy : {test_accuracy}")

313/313 [==============================] - 9s 28ms/step - loss: 1.0125 - accuracy: 0.7749
Test accuracy : 0.7749000191688538


In [5]:
import numpy as np

deltas = []
prev_weights = []
for epoch, weights in weights_dict.items():
    if(epoch == 0):
        prev_weights = np.array(weights, dtype=object)
        continue
    deltas.append((np.array(weights, dtype=object) - prev_weights)/prev_weights)
    prev_weights = np.array(weights, dtype=object)

deltas = np.array(deltas)
epochIdx = 0

for epoch_deltas in deltas:
    epoch_delta_sum = 0
    epoch_delta_size = 0
    for layer_deltas in epoch_deltas:
        epoch_delta_sum += (layer_deltas > 0.009).sum()
        epoch_delta_size += np.size(layer_deltas)
    print("Epoch:", epochIdx, "- {:.3f}".format(epoch_delta_sum/epoch_delta_size))
    epochIdx += 1

NameError: name 'np' is not defined

In [ ]:
# Checkpoint Model at Each Epoch and Selectively Load Weights